# Cohort 04 MLOPs-Zoomcap 2023

In [1]:
import pickle
import pandas as pd

Load the "model.bin" pickel file

In [3]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

K:\Anaconda3\lib\site-packages\sklearn\base.py:299: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.2.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
K:\Anaconda3\lib\site-packages\sklearn\base.py:299: UserWarning: Trying to unpickle estimator LinearRegression from version 1.2.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Process the data

In [4]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

Run this notebook for the February 2022 data.

In [27]:
year = '2022'
month= '02'

In [28]:
df = read_data(f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year}-{month}.parquet')

In [29]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

### Q01 Notebook
**What's the standard deviation of the predicted duration for this dataset?**

In [30]:
df_results= pd.DataFrame()
df_results['predicted_duration'] = y_pred
df_results['predicted_duration'].std()

5.281404481465004

Preparing the output

In [35]:
df_results['ride_id'] = f'{int(year):04d}/{int(month):02d}_' + df.index.astype('str')

In [36]:
output_file = f'{int(year):04d}-{int(month):02d}.parquet'

df_results.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

### Q02 Preparing the output
**What's the size of the output file?**

In [46]:
import os

print(os.stat(output_file).st_size/1024)

58588.8017578125


### Q03 Creating the scoring script
**Which command you need to execute for that?**

In [ ]:
!jupyter nbconvert --to script cohort04.ipynb